### Model 
- Training
    backbone(CLIP) + Dropout + Dense(units = 256) + Arcface + Softmax (classes = 17691)
- Inference
    backbone(CLIP) + Dropout + Dense(units = 256) + AdaptiveAveragePooling(n=64)

In [ ]:
from transformers import CLIPProcessor, TFCLIPVisionModel, CLIPFeatureExtractor

import re
import os 
import glob
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import random 
import math 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers 
from sklearn import metrics 
from sklearn.model_selection import KFold, train_test_split, StratifiedKFold
from tensorflow.keras import backend as K
import tensorflow_addons as tfa
from tqdm.auto import tqdm
from sklearn.preprocessing import normalize

import pickle
import json
import tensorflow_hub as tfhub
from datetime import datetime
import gc
import requests
from mpl_toolkits import axes_grid1

#### Device

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print("Running on TPU", tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else: 
    # Default TF works on CPU and GPU
    strategy = tf.distribute.get_strategy()
    
AUTO = tf.data.experimental.AUTOTUNE
print("Replicas: ", strategy.num_replicas_in_sync)

In [ ]:
# If GPU instance it makes mixed precision enable 
if strategy.num_replicas_in_sync == 1:
    from tensorflow.keras.mixed_precision import experimnetal as mix_precision
    policy = mixed_precition.Policy('mixed_float16')
    mixed_precision.set_policy(policy)
    


In [ ]:
class config:
    VERSION = 3
    SUBV = "Clip_ViT_Train"
    
    SEED = 42
    
    # pretrained Model
    RESUME = True
    RESUME_EPOCH = 0
    RESUME_WEIGHT = "../input/guei-v6-clip-vit-large-arcface-train-projection/clip-vit-large-patch14_224pix-emb256_arcface_entire.h5"
    
    #backbone model 
    model_type = "clip-vit-large-patch14"
    EFF_SIZE = 0
    EFF2_TYPE = ""
    IMAGE_SIZE = 224
    
    # projection layer 
    N_CLASSES = 17691
    EMB_DIM = 256
    
    # training 
    TRAIN = False
    BATCH_SIZE = 200 * strategy.num_replicas_in_sync
    EPOCHS = 100
    LR = 0.001
    save_dir = "./"
    
    DEBUG = False
    
# Function to seed everything 
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)

# model name 
MODEL_NAME = None
if config.model_type == "effnetv1":
    MODEL_NAME = f"effnet_v1_b{config.EFF_SIZE}"
elif config.model_type == "effnetv2":
    MODEL_NAME = f"effnetv2_{config.EFF2_TYPE}"
elif "swin" in config.model_type:
    MODEL_NAME = config.model_type
elif "conv" in config.model_tpye:
    MODEL_NAME = config.model_type
else:
    MODEL_NAME = config.model_type
    
config.MODEL_NAME = MODEL_NAME
print(MODEL_NAME)


### TFRecords

In [ ]:
from kaggle_datasets import KaggleDatasets

In [ ]:
train_shard_suffix = '*-train-*.tfrec'

ROOT_DIRS